In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/pheinemeyer/miniconda3/envs/road-emb/lib/python3.9/site-packages/IPython/core/extensions.py", line 76, in load_extension
  File "/home/pheinemeyer/miniconda3/envs/road-emb/lib/python3.9/site-packages/IPython/core/extensions.py", line 92, in _load_extension
  File "/home/pheinemeyer/miniconda3/envs/road-emb/lib/python3.9/importlib/__init__.py", line 127, in import_module
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 984, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'autoreload'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pheinemeyer/miniconda3/envs/road-emb/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
  File "/tmp/ipykernel_22625/2852571603.py", line 1, in <cell line: 1>
   

In [6]:
df = pd.read_csv("../datasets/trajectories/Porto/train.csv")
def convert_to_line_string(x):
    m = np.matrix(x).reshape(-1, 2)
    if m.shape[0] >= 10:
        line = LineString(m)
        return line
    return -1

df = df[df["MISSING_DATA"] == False]
df["POLYLINE"] = df["POLYLINE"].swifter.apply(convert_to_line_string)
df = df[df["POLYLINE"] != -1]

df.to_csv("../datasets/trajectories/Porto/mapped.csv")

In [ ]:
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")

In [ ]:
gdf.head()
gdf["POLYLINE"]

In [ ]:
gdf["coords"] = gdf["POLYLINE"].swifter.apply(lambda x: list(x.coords))

In [ ]:
gdf.head()

In [ ]:
# Folium Heat Map
from folium import plugins
from folium.plugins import HeatMap
from collections import defaultdict
import branca.colormap

map_porto = folium.Map(location=[41.18049, -8.64599],
                    zoom_start = 8)

coords = gdf.iloc[:, -1].values
print(coords.shape)
# cut down to 1/2 random choice, since full data is too big
indices = np.random.choice(coords.shape[0], int(coords.shape[0]/10))
reduced_coords = coords[indices]
print(reduced_coords.shape)
coords = np.concatenate(reduced_coords)
print(coords.shape)

heat_data = [(c[1], c[0]) for c in coords]

steps=20
colormap = branca.colormap.linear.viridis.scale(0, 1).to_step(steps)
colormap.caption = 'Density of GPS points with 10m radius'
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
colormap.add_to(map_porto)

HeatMap(heat_data, radius=10, gradient=gradient_map).add_to(map_porto)


# Display the map
#map_porto.save("heatmap_gps_points_porto.html")
map_porto

In [ ]:
map_porto.save('heatmap_gps_sample.html')